In [84]:
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
from typing import Tuple

# Project Idea: Find the (Non-Complex) Eigenvalue of a Matrix

It's from scratch-ish with some use of `numpy` because I don't want to write a hundred for-loops.

## References

* Lots of videos by **Mu Prime Math** ([used playlist](https://www.youtube.com/playlist?list=PLug5ZIRrShJHNCfEiX6l5CKbljWayGEcs))
* [Also this video](https://www.youtube.com/watch?v=LHlg_lfihiA)

## Power Method

An iterative algorithm that finds the eigenvalue with the highest magnitude.

$$ b_{k+1} = \frac{Ab_k}{||Ab_k||} $$

such that

* $A$ is an $n\times n$ matrix
* $b_k$ is an $n$-element vector
* $||Ab_k||$ is the element with the highest magnitude of $Ab_k$
* $b_0$ is non-zero
* Stop when $b_k = b_{k+1}$ (once the scaled $b_{k+1}$ is the same as $b_k$)

#### Why does it work?

Let's say we have matrix $A$ with eigenvectors $\vec{v_1}, \vec{v_2}, \ldots, \vec{v_n}$ with 
corresponding eigenvalues $\lambda_1, \lambda_2, \ldots, \lambda_n$.

Let $\vec{x}$ be written as a linear combination of the eigenvectors of $A$.

$$ \vec{x} = c_1\vec{v_1} + c_2\vec{v_2} + \ldots + c_n\vec{v_n} $$

We can then multiply $A$ to both sides, giving

$$ A\vec{x} = Ac_1\vec{v_1} + Ac_2\vec{v_2} + \ldots + Ac_n\vec{v_n} $$

If we keep multiplying $A$ to both sides (let's say $k$ times), we will end up with

$$ A^k\vec{x} = A^kc_1\vec{v_1} + A^kc_2\vec{v_2} + \ldots + A^kc_n\vec{v_n} $$

Because $\vec{v_1}, \vec{v_2}, \ldots, \vec{v_n}$ are eigenvectors of $A$, we know that $A\vec{v_i} = \lambda_i\vec{v_i} $.
We also know that if $\lambda$ is an eigenvalue of $A$, $\lambda^k$ would be the eigenvalue of $A^k$.
Thus, we get

$$ A^k\vec{x} = c_1\lambda_1^k\vec{v_1} + c_2\lambda_2^k\vec{v_2} + \ldots + c_n\lambda_n^k\vec{v_n} $$

Let $\lambda_1$ be the eigenvalue of $A$ with the highest magnitude. We can factor it out to get

$$ A^k\vec{x} = \lambda_1^k\left(c_1\vec{v_1} + \frac{c_2\lambda_2^k\vec{v_2}}{\lambda_1^k} + \ldots + \frac{c_n\lambda_n^k\vec{v_n}}{\lambda_1^k}\right) $$

Because $\lambda_1 > \lambda_2, \ldots, \lambda_n$,

$$\lim_{k\to\infty} \frac{\lambda_i^k}{\lambda_1^k} = 0~\forall i\neq1 $$

Therefore, we will end up with this when $k$ is very big

$$ A^k\vec{x} =  c_1 \lambda_1^k \vec{v_1} $$

This means that we can just choose an arbitrary $\vec{x}$ to find the eigenvalue with the
highest magnitude and its corresponding eigenvector as long as we multiply $A$ enough times.

We will end up with a scaled version of an eigenvector, denoted as $c\vec{v}$ for some constant $c$.

Finally, we can find the eigenvalue. We can do so by multiplying $c\vec{v}$ by $A$ and find out how
much $c\vec{v}$ is scaled by the multiplication because $A\vec{v} = \lambda{v}$ and that would be 
our corresponding eigenvalue.

In [85]:
def power_eig(A:np.ndarray) -> Tuple[float, np.ndarray]:
	size, _ = A.shape
	A = A.astype(float)
	vec = np.ones(size).astype(float)
	pvec = np.zeros(size).astype(float)

	while (vec != pvec).all():
		pvec = vec
		imax = np.argmax(np.abs(vec))
		vec = np.matmul(A, vec) / vec[imax]

	i = np.argmax(np.abs(vec))
	return vec[i], vec/vec[i]

In [86]:
test_matrix = np.array([[5., 8., 16.],
						[4., 1., 8.],
						[-4., -4., -11.]])
max_eival, max_eivec = power_eig(test_matrix)
print(max_eival, max_eivec)

-3.0000000000000018 [ 1.          0.42857143 -0.71428571]


In [87]:
print("eigenvalue", max_eivec)
print("A*eivec", np.matmul(test_matrix, max_eivec))
print("eival*eivec", max_eival * max_eivec)

eigenvalue [ 1.          0.42857143 -0.71428571]
A*eivec [-3.         -1.28571429  2.14285714]
eival*eivec [-3.         -1.28571429  2.14285714]


## Shifted Inverse Power Method

Finding eigenvalues that isn't the biggest one.

The problem with the power method is that it only works for finding the eigenvalue and eigenvector
for the eigenvalue that has the highest magnitude due to it *dominating* the other terms as shown
above.

Thus, we have to modify the power method so that it works with finding the other eigenvalues of the matrix.

In [88]:
def shifted_ip_eig(A:np.ndarray, alpha:float, k:int=100):
	size, _ = A.shape
	vec = np.ones(size)

	A = A.astype(float)
	vec = vec.astype(float)
	inv = np.linalg.inv(A - alpha*np.identity(size))

	for i in range(k):
		vec = np.matmul(inv, vec)
		coeff = max(vec)
		vec = vec / coeff

	last = np.matmul(A, vec)
	return np.average(last/vec), vec

In [89]:
eival, eivec = shifted_ip_eig(test_matrix, 1.5)
print(eival, eivec)

0.9999999999999979 [ 1.   0.5 -0.5]


In [90]:
eivec, np.matmul(test_matrix, eivec) # A*v = 1*v

(array([ 1. ,  0.5, -0.5]), array([ 1. ,  0.5, -0.5]))